<a href="https://colab.research.google.com/github/ggoni/cryptoAnalisis/blob/main/CryptoFCST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import numpy as np

from datetime import datetime

import matplotlib.pyplot as plt
from bokeh.layouts import column
from bokeh.plotting import figure, show
from bokeh.layouts import column,row
from bokeh.models import HoverTool
from bokeh.io import output_notebook
from dateutil.relativedelta import relativedelta
from fbprophet import Prophet


output_notebook()

In [ ]:

%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [ ]:
def get_prices(symbol,cur='CLP',api_key = api_key):

    url = 'https://min-api.cryptocompare.com/data/v2/histoday?fsym='+symbol+'&tsym='+\
        cur+'&limit=800&api_key='+api_key

    df = pd.DataFrame(requests.get(url).json()["Data"]["Data"])
    df = df.iloc[:,:-2] 
    
    df = df[::-1]
    df['date']=df.time.apply(datetime.fromtimestamp)
    #df = df[df.date.dt.weekday==0]
    df.reset_index(inplace=True,drop=True)
    df['diff']=df.close-df.close.shift(-4)
    df['flip']=df['diff'][::-1].apply(lambda x: 1 if x<0 else 0)
    df['DM']=df.flip[::-1].rolling(9).sum()
    largo=min(len(df),30)
    df['prom']=df.close[::-1].rolling(largo).mean()
    df['desvest']=df.close[::-1].rolling(largo).std()
    df['bolup']=df.prom+2.*df.desvest
    df['boldown']=df.prom-2.*df.desvest
    
    
    return df

In [ ]:
def predice(symbol, cur = 'CLP',api_key=api_key):
    df = get_prices(symbol)
    df['ds'] = df.date 
    df ['y'] = df.close
    df = df [['ds','y']]
    model = Prophet()
    model.fit(df)
    future = model.make_future_dataframe(periods=7)
    forecast = model.predict(future)
    
    model.plot(forecast.iloc[-60:])
    return forecast[['ds','yhat']].iloc[-8:-5]

In [ ]:
def rCiclo(symbol,p=5):
    df=get_prices(symbol=symbol)
    revisar=df.copy()
    hovern = HoverTool(tooltips=[('Último:', '@close'),('DeMark:', '@DM')])
    p1=figure(x_axis_type='datetime',title='Evolución valor Par '+symbol,
              tools=[hovern,'save','wheel_zoom'])
    p1.line(x='date',y='close',source=revisar,line_width=3,color='#110146')
    p1.yaxis.axis_label = 'Precio'
    p1.line(x='date',y='bolup',source=revisar,color='#FF6200',line_dash='dashed',
            alpha=0.85)
    p1.line(x='date',y='boldown',source=revisar,color='#FF6200',line_dash='dashed',
            alpha=0.85)
    show(p1)
    
    valores = np.log(df.close/df.close.shift(-1))[:-1]
    
    hist, edges = np.histogram(valores, density=True, bins=100)
    valor_prcntl= np.percentile(valores[:-1],p)   
    valor_prcnt2= np.percentile(valores[:-1],100-p)
    
    p2=figure(title='Histograma retornos diarios '+symbol,
              tools=[hovern,'save','wheel_zoom'])
    
    
    p2.quad(top=hist, bottom=0.0, left=edges[:-1], right=edges[1:], line_color="white",alpha=0.5)


    

    
    x1 = [valor_prcntl,valor_prcntl]
    y1 = [0,np.max(hist)]
    p2.line(x1,y1,line_width=2,color='red',line_dash='dotted',legend_label="p"+str(p)+" Variación")
    
    valor_hoy = np.log(df.close[0]/df.close[1])
    
    x2 = [valor_hoy,valor_hoy]
    y2 = [0,np.max(hist)]
    p2.line(x2,y2,line_width=2,color='grey', line_dash='solid',legend_label="Variación Hoy")
    
    
    x3 = [valor_prcnt2,valor_prcnt2]
    y3 = [0,np.max(hist)]
    p2.line(x3,y3,line_width=2,color='green',line_dash='dotted',legend_label="p"+str(100-p)+" Variación")
    
    
    
    
    show(p2)


In [ ]:
rCiclo('ETH')

In [ ]:
rCiclo('BTC')

In [ ]:
rCiclo('LTC')

In [ ]:
rCiclo('BCH')